# Atelier #2 - Carvana Image Masking Challenge avec PyTorch
### Automatically identify the boundaries of the car in an image

![](static/header.png)

Chaque noteobook commence par ces trois lignes; ils garantissent que toutes les modifications apportées aux modules et paquets que vous développez "en dehors" du Jupyter Notebook sont automatiquement rechargées lorsque modifiés. Tous les graphiques et images sont affichées.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Chargement des librairies

In [ ]:
import os
from tqdm.notebook import tqdm
#from tqdm import tqdm

from IPython.core.display import Image, display
from PIL import Image as Img

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch import optim
from torch.utils.tensorboard import SummaryWriter

from eval import eval_net

from utils.dataset import BasicDataset

---
## Vérification de CUDA

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

<h2 style="color: red">Pour la suite de l'exercice, la mémoire du GPU peut-être libérée en redémarrant le kernel</h2>

---
## Répertoires des données

Récupérez les données sur le site [Kaggle](https://www.kaggle.com/c/carvana-image-masking-challenge)

In [ ]:
IMAGES = '<completer>'
MASKS = '<completer>'
CHECKPOINTS = 'checkpoints'

### Image non segmentée

In [ ]:
img = IMAGES + '00087a6bd4dc_01.jpg'
display(Img.open(img))

### Masque

In [ ]:
msk = MASKS + '00087a6bd4dc_01_mask.gif'
display(Img.open(msk))

---
## Hyper-paramètres d'apprentissage

In [ ]:
# Nombre d'époques
epochs = 1
# Taille de la bacth
batch_size = 4
# Facteur d'apprentissage
lr = 0.001
# Facteur de redimensionnement des images (downscaling)
scale = 0.1

In [ ]:
# Pourcentage de données de validation
validation_ratio = 0.5
# Active la sauvegarde par checkpoints
save_cp = True

---
## Le modèle de type U-Net

![](static/unet-pub.png)

![](static/architecture-unet.png)

In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [ ]:
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [ ]:
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [ ]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        # TODO: Compléter cette méthode pour obtenir l'architecture UNet présentée ci-dessus
        # ~ 10 lignes
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        
        return logits

In [ ]:
net = UNet(n_channels=3, n_classes=1)

---
## Préparation de l'entraînement

In [ ]:
model = None #"checkpoints/CP_epoch1.pth" #None # Remplacer par votre modèle pré-entraîné

if model is not None:
    net.load_state_dict(torch.load(model, map_location=device))
    print(f'Model loaded from {model}')

In [ ]:
net.to(device=device)
torch.backends.cudnn.benchmark = True  # faster convolutions, but more memory

In [ ]:
dataset = BasicDataset(IMAGES, MASKS, scale)
n_val = int(len(dataset) * validation_ratio)
n_train = len(dataset) - n_val
torch.manual_seed(2019)
train, val = random_split(dataset, [n_train, n_val] )
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

Vérification des paramètres d'entraînement

In [ ]:
print(f"""Starting training:
    Epochs:          {epochs}
    Batch size:      {batch_size}
    Learning rate:   {lr}
    Training size:   {n_train}
    Validation size: {n_val}
    Device:          {device.type}
    Images scaling:  {scale}""")

In [ ]:
# Setup Tensorboard
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{scale}')
global_step = 0
optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8)
if net.n_classes > 1:
    criterion = nn.CrossEntropyLoss()
else:
    criterion = nn.BCEWithLogitsLoss()

---
## Entraînement

In [ ]:
for epoch in range(epochs):
    net.train()

    epoch_loss = 0
    with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
        for batch in train_loader:
            imgs = batch['image']
            true_masks = batch['mask']

            imgs = imgs.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.float32)

            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            epoch_loss += loss.item()
            writer.add_scalar('Loss/train', loss.item(), global_step)

            pbar.set_postfix(**{'loss (batch)': loss.item()})

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.update(imgs.shape[0])
            global_step += 1
            if global_step % (len(dataset) // (10 * batch_size)) == 0:
                val_score = eval_net(net, val_loader, device, n_val)
                if net.n_classes > 1:
                    print('Validation cross entropy: {}'.format(val_score))
                    writer.add_scalar('Loss/test', val_score, global_step)

                else:
                    print('Validation Dice Coeff: {}'.format(val_score))
                    writer.add_scalar('Dice/test', val_score, global_step)

                writer.add_images('images', imgs, global_step)
                if net.n_classes == 1:
                    writer.add_images('masks/true', true_masks, global_step)
                    writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

    if save_cp:
        try:
            os.mkdir(CHECKPOINTS)
            print('Created checkpoint directory')
        except OSError:
            pass
        torch.save(net.state_dict(), CHECKPOINTS + f'CP_epoch{epoch + 1}.pth')
        print(f'Checkpoint {epoch + 1} saved !')

# Fermeture du writer pour TensorBoard
writer.close()

---
## TensorBoard

Depuis un terminal, démarrez Tensorboard avec la commande. **Pensez à ouvrir le port 6006 (Firewall Rules)**

`tensorboard --host 0.0.0.0 --logdir=runs`

---
## Optionnel - 2018 Data Science Bowl

Essayez le U-Net avec les données du 2018 [Data Science Bowl](https://www.kaggle.com/c/data-science-bowl-2018)

---
## Références

* [Carvana Image Masking Challenge | Kaggle | Private leaderboard](https://www.kaggle.com/c/carvana-image-masking-challenge/leaderboard)
* [UNet: semantic segmentation with PyTorch](https://github.com/milesial/Pytorch-UNet)
* [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)